# Model Evaluation on Sentiment 140 Test Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/My Drive/Tilburg DSS/Thesis/Data
import pandas as pd

In [ ]:
pip install zipfile36

In [ ]:
#reference: https://github.com/Azure-Samples/MachineLearningSamples-TwitterSentimentPrediction
#import the dataset downloaded from Kaggle
import zipfile
import pandas as pd
with zipfile.ZipFile("trainingandtestdata.zip") as z:
    with z.open("testdata.manual.2009.06.14.csv") as f:
        colnames=['Polarity', 'X', 'Y', 'Z', 'T', "Tweet"]
        test = pd.read_csv(f, header=None, sep=',', encoding='latin-1', names=colnames, index_col=False)
      
    print(train.head()) 

In [ ]:
df=pd.DataFrame(test, columns=("Tweet", "Polarity"))

In [ ]:
df.Polarity.value_counts()

In [ ]:
labels = {0: 'negative', 2: 'neutral', 4: 'positive'}
df['Polarity'] = df['Polarity'].map(labels)

In [ ]:
df.to_pickle("Sentiment140Test.pkl")

# VADER

In [ ]:
!pip install vaderSentiment

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
# Initialize VADER so we can use it later
sentimentAnalyser = SentimentIntensityAnalyzer()

In [ ]:
import numpy as np
import pandas as pd
import nltk
#download vader from nltk
nltk.download('vader_lexicon')

In [ ]:
df['polarity_unproc'] = df.Tweet.apply(lambda x: sentimentAnalyser.polarity_scores(x))

# TEXTBLOB

In [ ]:
from textblob import TextBlob

In [ ]:
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity
  
 #Create a function to get the polarity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity
  

In [ ]:
df["TextBlob_Polarity_unproc"]=df["Tweet"].apply(lambda x: getPolarity(x))

In [ ]:
sentiment_VADER_Textblob=pd.DataFrame(data=df, columns=["Tweet", "Polarity", 'polarity_unproc', 'TextBlob_Polarity_unproc'])

In [ ]:
sentiment_VADER_Textblob["polarity_unproc"]=df.polarity_unproc.apply(lambda x: x["compound"])

In [ ]:
sentiment_VADER_Textblob.to_pickle("Sentiment140_test_VADER_Textblob.pkl")

In [ ]:
labels = {'negative': -1, 'neutral':0, 'positive':1}
sentiment_VADER_Textblob['Polarity'] = sentiment_VADER_Textblob['Polarity'].map(labels)

In [ ]:
def getScores(score):
    if score < 0:
        return -1
    elif score == 0:
        return 0
    else:
        return 1


In [ ]:
sentiment_VADER_Textblob["TextBlob_Polarity_unproc"] = sentiment_VADER_Textblob["TextBlob_Polarity_unproc"].apply(lambda x: getScores(x))

In [ ]:
def getScores2(score):
    if score >= 0.05 :
        return 1
    elif score <= -0.05:
        return -1
    else:
        return 0
 

In [ ]:
#Polarity_unproc: VADER model results on unprocessed data
sentiment_VADER_Textblob["polarity_unproc"] = sentiment_VADER_Textblob["polarity_unproc"].apply(lambda x: getScores2(x))

In [ ]:
sent=sentiment_VADER_Textblob

In [ ]:
df.to_pickle("Sentiment140_test_VADER_Textblob.pkl") #save the DataFrame as a pickle file:

# Evaluation Metrics

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
print(classification_report(sent['Polarity'], sent['polarity_unproc']))

In [ ]:
print(classification_report(sent['Polarity'], sent['TextBlob_Polarity_unproc']))

# BERT Models

In [ ]:
pip install -q transformers
from transformers import pipeline

In [ ]:
classifier = pipeline("sentiment-analysis",model="finiteautomata/bertweet-base-sentiment-analysis")
classifier2 = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest")

In [ ]:
df["BERTweet_unproc"]=df.Tweet.apply(lambda x: classifier(x))

df["Roberta_Latest_unproc"]=df.Tweet.apply(lambda x: classifier2(x))

In [ ]:
df.to_pickle("BERTs_Sent140_test.pkl")

# Combine Model Results & Evaluate

In [ ]:
df_140_VADER_Textblob=pd.read_pickle("Sentiment140_test_VADER_Textblob.pkl")

In [ ]:
df = pd.read_pickle("BERTs_Sent140_test.pkl")

In [ ]:
from functools import reduce  

data_merge1 = reduce(lambda left, right:  
                     pd.merge(left , right,
                              on = ['Tweet', 'Polarity']),
                     [df_140_VADER_Textblob, df])
Models=pd.DataFrame(data_merge1)

Models=Models.rename(columns={"polarity_unproc":"VADER_unproc"})

Models.columns

Models=Models[['Tweet', 'Polarity', 'VADER_unproc',
       'TextBlob_Polarity_unproc', 'BERTweet_unproc',
       'Roberta_Latest_unproc']]

preds2=[]
for i in Models.BERTweet_unproc:
    for j in i:
        if j["label"].startswith('P'):
            preds2.append(1)
        elif j["label"].startswith('NEU'):
             preds2.append(0)
        else:
             preds2.append(-1)

Models["BERTweet_unproc"]=pd.DataFrame(data=preds2)

labels = {'negative': -1, 'neutral':0, 'positive':1}
Models['Polarity'] = Models['Polarity'].map(labels)


preds4=[]
for i in Models.Roberta_Latest_unproc:
    for j in i:
        if j["label"].startswith('p'):
            preds4.append(1)
        elif j["label"].startswith('neu'):
            preds4.append(0)
        else:
            preds4.append(-1)

Models["Roberta_Latest_unproc"]=pd.DataFrame(data=preds4)

Models

def getScores(score):
    if score < 0:
        return -1
    elif score == 0:
        return 0
    else:
        return 1


Models["TextBlob_Polarity_unproc"]=Models["TextBlob_Polarity_unproc"].apply(lambda x: getScores(x))

def getScores2(score):
    if score >= 0.05 :
        return 1
    elif score <= -0.05:
        return -1
    else:
        return 0
    
Models["VADER_unproc"] = Models["VADER_unproc"].apply(lambda x: getScores2(x))

# Show the classification report

print(classification_report(Models['Polarity'], Models['VADER_unproc']))

print(classification_report(Models['Polarity'], Models["TextBlob_Polarity_unproc"]))

print(classification_report(Models['Polarity'], Models['Roberta_Latest_unproc']))

print(classification_report(Models['Polarity'], Models['BERTweet_unproc']))

